In [1]:
import numpy as np
import tensorflow as tf
from utils import load_dataset
from metrics import calculate_metrics

# Load test dataset
train_ds, test_ds = load_dataset()

# # Prepare test dataset for calculating FID
# test_ds[0] = test_ds[0].batch(32)
# test_ds[1] = test_ds[1].batch(32) 

2023-06-05 19:19:03.736431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 19:19:04.357647: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-06-05 19:19:04.357721: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-06-05 19:19:04.357727: W tensorflow/compiler/tf2tensorrt/utils/py_utils

/home/paperspace/Documents/code/models/cycleGAN/all/train_dataset_input


e NUMA node, so returning NUMA node zero
2023-06-05 19:19:05.743187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-05 19:19:05.743296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-05 19:19:06.272032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-05 19:19:06.272314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-05 19:19:06.272538:

In [28]:
count = 0
train_ds_len = 0
for img_idx, train_image_set in enumerate((tf.data.Dataset.zip((train_ds[0], train_ds[1])))):
    if count % 2 == 0:
        train_ds_len += 1
    count += 1
        
train_ds_len

1091

In [4]:
count = 0
test_ds_len = 0
for img_idx, test_image_set in enumerate((tf.data.Dataset.zip((test_ds[0], test_ds[1])))):
    if count % 2 == 0:
        test_ds_len += 1
    count += 1
        
test_ds_len

343

In [25]:
# Load model
model_path = '/home/paperspace/Documents/code/results/cycleGAN/all/06032023_14:03:31/gen_g.h5'
gen = tf.keras.models.load_model(model_path)

In [26]:
# Apply trained models on test dataset to calculate mean metrics
tf.print(f"Testing model performance on test dataset...")
metrics_names = ['SSIM', 'PSNR', 'RSME', 'MAE']
metrics = {'SSIM': [], 
        'PSNR': [], 
        'RSME': [], 
        'MAE': [],
        }

count = 1
for img_idx, test_image_set in enumerate((tf.data.Dataset.zip((test_ds[0], test_ds[1])))):
    if count % 2 == 0:
        fake_image_y = gen(test_image_set[0])
        calculated_metrics = calculate_metrics(tf.squeeze(fake_image_y, axis=0)[:,:,0], tf.squeeze(test_image_set[1], axis=0)[:,:,0])
        
        for metric_idx, metric in enumerate(calculated_metrics):
            metrics[metrics_names[metric_idx]].append(metric)

    count += 1

for metric in metrics_names:   
    mean_metric = np.mean(np.asarray(metrics[metric]))
    std_metric = np.std(np.asarray(metrics[metric]))
    tf.print(f"The mean {metric} is: {mean_metric:2f} +- {std_metric:2f}.")

Testing model performance on test dataset...
The mean SSIM is: 0.753317 +- 0.017314.
The mean PSNR is: 20.736832 +- 0.794593.
The mean RSME is: 0.184505 +- 0.016949.
The mean MAE is: 0.070676 +- 0.008590.


In [ ]:
import numpy as np
from scipy import linalg
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input

def calculate_fid(real_images, generated_images, batch_size=32):
    
    # Load InceptionV3 model without top classification layer
    inception_model = InceptionV3(include_top=False, pooling='avg')

    # Calculate activations for real and generated images
    real_activations = inception_model.predict(real_images, batch_size=batch_size)
    generated_activations = inception_model.predict(generated_images, batch_size=batch_size)

    # Calculate mean and covariance of real and generated activations
    mu_real = np.mean(real_activations, axis=0)
    mu_generated = np.mean(generated_activations, axis=0)
    sigma_real = np.cov(real_activations, rowvar=False)
    sigma_generated = np.cov(generated_activations, rowvar=False)

    # Calculate squared Frobenius norm between means
    mean_diff = mu_real - mu_generated
    mean_squared_norm = np.sum(mean_diff**2)

    # Calculate trace of the product of covariances
    cov_product = linalg.sqrtm(sigma_real.dot(sigma_generated))
    if np.iscomplexobj(cov_product):
        cov_product = cov_product.real

    # Calculate FID
    fid = mean_squared_norm + np.trace(sigma_real + sigma_generated - 2 * cov_product)

    return fid

In [ ]:
mean_FID = 0

for idx, test_image in enumerate(tf.data.Dataset.zip((test_ds[0], test_ds[1]))):
    FID = calculate_fid
    mean_FID += FID
    
mean_FID = mean_FID/idx